In [ ]:
import sys

sys.path.append('D:\PythonProjects\\bk-monitor\\venv2\share\scripts')
import hello

In [ ]:
"""
新增策略排查脚本，排查策略关联的通知组在业务下不存在的策略。(通知组就是告警组)

遍历全部策略， 查看策略配置的通知组id，是否在当前业务下存在。 
    -如果查询不到， 则去掉业务过滤条件查询， 
    -如果存在，则将通知组的名称作为查询条件，在当前业务下查询。（查询当前业务下相同名称的告警组）
    如果有记录， 则替换为当前查询到的通知组id。并打印。
    
替换功能可以通过一个配置项（是否预览）开启。
开启预览，则仅打印信息不替换。 不开启预览，则替换+打印信息。

策略id， 策略名称， 业务id， 关联的通知组id+对应的业务id，本业务下相同名称的通知组id
"""

In [146]:
from bkmonitor.models.strategy import StrategyModel, UserGroup
from bkmonitor.models.fta.action import StrategyActionConfigRelation
from pprint import pprint


def create_user_group():
    """
    创建告警组
    """
    # 清空数据
    UserGroup.objects.all().delete()
    user_group_ids = []
    for i in range(1, 6):
        instance = UserGroup.objects.create(name=f"test_user_group_{i}", bk_biz_id=2, desc=f"test_user_group_{i}_desc")
        user_group_ids.append(instance.id)

    # 创建具有相同名称的告警组
    for i in range(1, 6):
        instance = UserGroup.objects.create(name=f"test_user_group_{i}", bk_biz_id=2, desc=f"test_user_group_{i}_desc")
        user_group_ids.append(instance.id)

    # 创建非当前业务下的告警组
    for i in range(1, 6):
        instance = UserGroup.objects.create(name=f"test_user_group_{i}", bk_biz_id=6, desc=f"test_user_group_{i}_desc")
        user_group_ids.append(instance.id)

    for i in range(1, 6):
        instance = UserGroup.objects.create(name=f"test_user_group_{i}", bk_biz_id=5, desc=f"test_user_group_{i}_desc")
        user_group_ids.append(instance.id)

    return user_group_ids


def create_strategy():
    """
    创建策略
    """
    # 清空数据  
    StrategyModel.objects.all().delete()

    strategy_ids = []
    for i in range(1, 21):
        instance = StrategyModel.objects.create(name=f"test_stragety_{i}", bk_biz_id=2)
        strategy_ids.append(instance.id)
    return strategy_ids


# 创建告警组
def create_strategy_action_config():
    StrategyActionConfigRelation.objects.all().delete()

    # 创建告警组
    user_group_ids = create_user_group()
    # 创建新的策略
    stragety_ids = create_strategy()
    for i in range(len(stragety_ids)):
        StrategyActionConfigRelation.objects.create(config_id=0, strategy_id=stragety_ids[i],
                                                    user_groups=user_group_ids[i:i + 2])

    # 打印关联信息
    query_set = StrategyActionConfigRelation.objects.all().values("id", "strategy_id", "user_groups")
    for item in query_set:
        user_groups = UserGroup.objects.filter(id__in=item["user_groups"]).values("id", "bk_biz_id")
        strategy = StrategyModel.objects.get(id=item["strategy_id"])
        item["user_groups_info"] = user_groups
        item["strategy_name"] = strategy.name
        pprint(item)


create_strategy_action_config()

{'id': 1243,
 'strategy_id': 1253,
 'strategy_name': 'test_stragety_1',
 'user_groups': [508, 509],
 'user_groups_info': <QuerySet [{'id': 509, 'bk_biz_id': 2}, {'id': 508, 'bk_biz_id': 2}]>}
{'id': 1244,
 'strategy_id': 1254,
 'strategy_name': 'test_stragety_2',
 'user_groups': [509, 510],
 'user_groups_info': <QuerySet [{'id': 510, 'bk_biz_id': 2}, {'id': 509, 'bk_biz_id': 2}]>}
{'id': 1245,
 'strategy_id': 1255,
 'strategy_name': 'test_stragety_3',
 'user_groups': [510, 511],
 'user_groups_info': <QuerySet [{'id': 511, 'bk_biz_id': 2}, {'id': 510, 'bk_biz_id': 2}]>}
{'id': 1246,
 'strategy_id': 1256,
 'strategy_name': 'test_stragety_4',
 'user_groups': [511, 512],
 'user_groups_info': <QuerySet [{'id': 512, 'bk_biz_id': 2}, {'id': 511, 'bk_biz_id': 2}]>}
{'id': 1247,
 'strategy_id': 1257,
 'strategy_name': 'test_stragety_5',
 'user_groups': [512, 513],
 'user_groups_info': <QuerySet [{'id': 513, 'bk_biz_id': 2}, {'id': 512, 'bk_biz_id': 2}]>}
{'id': 1248,
 'strategy_id': 1258,
 'str

In [3]:
from bkmonitor.management.commands.run_cmd import parse_usergroup_not_under_business

parse_usergroup_not_under_business(True)

策略id，策略名称，业务id，关联的通知组id列表，对应的业务id列表，本业务下相同名称的通知组id字典
1262,test_stragety_10,2,[517 518],[2 6],{'test_user_group_1': [513, 508]}
1263,test_stragety_11,2,[518 519],[6],{'test_user_group_1': [513, 508], 'test_user_group_2': [514, 509]}
1264,test_stragety_12,2,[519 520],[6],{'test_user_group_2': [514, 509], 'test_user_group_3': [515, 510]}
1265,test_stragety_13,2,[520 521],[6],{'test_user_group_4': [516, 511], 'test_user_group_3': [515, 510]}
1266,test_stragety_14,2,[521 522],[6],{'test_user_group_4': [516, 511], 'test_user_group_5': [517, 512]}
1267,test_stragety_15,2,[522 523],[5 6],{'test_user_group_5': [517, 512], 'test_user_group_1': [513, 508]}
1268,test_stragety_16,2,[523 524],[5],{'test_user_group_1': [513, 508], 'test_user_group_2': [514, 509]}
1269,test_stragety_17,2,[524 525],[5],{'test_user_group_2': [514, 509], 'test_user_group_3': [515, 510]}
1270,test_stragety_18,2,[525 526],[5],{'test_user_group_4': [516, 511], 'test_user_group_3': [515, 510]}
1271,test_stragety_19,2,[526 52

In [4]:
parse_usergroup_not_under_business(False)

策略id，策略名称，业务id，相同名称的告警组id列表，对应的业务id列表，本业务下相同名称的通知组id字典
1262,test_stragety_10,2,[513 508] [2],{'test_user_group_1': [513, 508]}
1263,test_stragety_11,2,[513 508 514 509] [2],{'test_user_group_1': [513, 508], 'test_user_group_2': [514, 509]}
1264,test_stragety_12,2,[514 509 515 510] [2],{'test_user_group_2': [514, 509], 'test_user_group_3': [515, 510]}
1265,test_stragety_13,2,[516 511 515 510] [2],{'test_user_group_4': [516, 511], 'test_user_group_3': [515, 510]}
1266,test_stragety_14,2,[516 511 517 512] [2],{'test_user_group_4': [516, 511], 'test_user_group_5': [517, 512]}
1267,test_stragety_15,2,[517 512 513 508] [2],{'test_user_group_5': [517, 512], 'test_user_group_1': [513, 508]}
1268,test_stragety_16,2,[513 508 514 509] [2],{'test_user_group_1': [513, 508], 'test_user_group_2': [514, 509]}
1269,test_stragety_17,2,[514 509 515 510] [2],{'test_user_group_2': [514, 509], 'test_user_group_3': [515, 510]}
1270,test_stragety_18,2,[516 511 515 510] [2],{'test_user_group_4': [516, 511], 't